<a href="https://colab.research.google.com/github/bluemango0312/lettrip-app/blob/main/%EC%95%84%EC%9D%B4%ED%85%9C%EA%B8%B0%EB%B0%98%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 로드 및 전처리
csv_path = '/content/sample_data/data_60.csv'
data = pd.read_csv(csv_path, encoding='utf-8')

rating_mean = data['rating'].mean()
data['rating_normalized'] = data['rating'] - rating_mean

user_ids = data['userId'].unique()
item_ids = data['placeName'].unique()

user_id_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
item_id_to_idx = {item_id: idx for idx, item_id in enumerate(item_ids)}

num_users = len(user_ids)
num_items = len(item_ids)
rating_matrix = np.zeros((num_items, num_users))

for _, row in data.iterrows():
    user_idx = user_id_to_idx[row['userId']]
    item_idx = item_id_to_idx[row['placeName']]
    rating_matrix[item_idx, user_idx] = row['rating_normalized']

item_similarity = cosine_similarity(rating_matrix)

# 추천 모델 구성 - 케라스 Model 클래스 상속
class ItemBasedRecommender(keras.Model):
    def __init__(self, num_items, item_similarity):
        super(ItemBasedRecommender, self).__init__()
        self.item_similarity = self.add_weight("item_similarity", shape=(num_items, num_items), initializer=tf.keras.initializers.Constant(item_similarity), trainable=False)

    def call(self, inputs):
        user_idx = inputs  # Remove the unpacking
        user_ratings = tf.gather(self.item_similarity, user_idx)
        recommendations = tf.linalg.matmul(self.item_similarity, tf.expand_dims(user_ratings, axis=-1))
        return tf.squeeze(recommendations, axis=-1)


# 모델 생성
model = ItemBasedRecommender(num_items, item_similarity)

# 모델 학습
X_train = np.array([user_id_to_idx[user_id] for user_id in user_ids])
y_train = rating_matrix.T  # 실제 평점 행렬을 사용하여 학습

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

# 예측 결과 및 유사도 출력
user_id = 10441490830
test_user_idx = user_id_to_idx.get(user_id, None)
if test_user_idx is not None:
    recommendations2 = model([test_user_idx])
    sorted_item = np.argsort(recommendations2)[::-1][:10]

    print(f"Recommended items for user {user_id}:")
    for item_idx in sorted_item:
        item_id = item_ids[item_idx]
        similarity_score = item_similarity[item_idx, test_user_idx].item()  # .item() 추가
        print(f"Item: {item_id}, Similarity Score: {similarity_score:.4f}")
else:
    print(f"User {user_id} not found in the data.")


# 모델 평가
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
print(f"MAE: {mae:.4f}, MSE: {mse:.4f}")



Epoch 1/10
2/2 [==============================] - 9s 4s/step - loss: 932.7805
Epoch 2/10
2/2 [==============================] - 10s 5s/step - loss: 932.7805
Epoch 3/10
2/2 [==============================] - 10s 5s/step - loss: 932.7805
Epoch 4/10
2/2 [==============================] - 10s 4s/step - loss: 932.7805
Epoch 5/10
2/2 [==============================] - 10s 5s/step - loss: 932.7805
Epoch 6/10
2/2 [==============================] - 10s 5s/step - loss: 932.7805
Epoch 7/10
2/2 [==============================] - 10s 5s/step - loss: 932.7805
Epoch 8/10
2/2 [==============================] - 10s 5s/step - loss: 932.7805
Epoch 9/10
2/2 [==============================] - 10s 5s/step - loss: 932.7805
Epoch 10/10
2/2 [==============================] - 10s 5s/step - loss: 932.7805
Recommended items for user 10441490830:


ValueError: ignored